Installing

In [ ]:
!pip install bitsandbytes
!pip install transformers==4.34.0
!pip install -U peft
!pip install accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install wandb

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from peft import PeftModel
import torch
import os

In [ ]:
HUGGING_FACE_HUB_TOKEN=""
wandb=""
access_token = HUGGING_FACE_HUB_TOKEN
login(token=access_token)
!huggingface-cli login --token $secret_hf

### Merging the adopter

### Load base model with adopter

In [ ]:
base_model = "mistralai/Mistral-7B-v0.1"
new_model = "Jyotiyadav/copywriting"

In [ ]:
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= "auto")

model = PeftModel.from_pretrained(base_model_reload, new_model)

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 520097664 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 536879104 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


### Reload tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Inference

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [ ]:
prompt = """
As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).

    ### Input data:
    Cookies

    ### Output:
    {
    'ingredients': "['Farina 00', 'Burro', 'Bicarbonato', 'Uova', 'Zucchero di canna', 'Zucchero', 'Gocce di cioccolato fondente', 'Sale fino']",
    'description': 'These "Cookies" are a delightful treat, classically made with fine "Farina 00," rich "Burro," leavening "Bicarbonato," "Uova" (eggs), and a blend of "Zucchero di canna" and "Zucchero" (sugars) to create a perfectly sweet foundation. Indulgent "Gocce di cioccolato fondente" (chocolate chips) are generously mixed in for added luxury, while a pinch of "Sale fino" (fine salt) heightens all the flavors.',
    'allergen': "['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Peanuts', 'SOYBEAN', 'Latte', 'Nuts', 'Celery', 'Mustard', 'Sesame seeds', 'Sulfur dioxide and sulphites', 'Shell', 'Clams']",
    'Additional_information': '[]'
    }
"""

In [ ]:
sequences = pipe(
    f"<s>[INST] {prompt} [/INST]",
    do_sample=True,
    max_new_tokens=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] 
<s>[INST]As a restaurant menu manager, your role is to gather below informations based on input  (Name of the dish).
    generate the output

    ### Input data:Cookies

    ### Output:
    {
    'ingredients': "['Farina 00', 'Burro', 'Bicarbonato', 'Uova', 'Zucchero di canna', 'Zucchero', 'Gocce di cioccolato fondente', 'Sale fino']",
    'description': 'These "Cookies" are a delightful treat, classically made with fine "Farina 00," rich "Burro," leavening "Bicarbonato," "Uova" (eggs), and a blend of "Zucchero di canna" and "Zucchero" (sugars) to create a perfectly sweet foundation. Indulgent "Gocce di cioccolato fondente" (chocolate chips) are generously mixed in for added luxury, while a pinch of "Sale fino" (fine salt) heightens all the flavors.',
    'allergen': "['Cereals', 'Crustaceans', 'Egg', 'Fish', 'Peanuts', 'SOYBEAN', 'Latte', 'Nuts', 'Celery', 'Mustard', 'Sesame seeds', 'Sulfur dioxide and sulphites', 'Shell', 'Clams']",
    'Additional_information': '[]'
    }

### Pushing the fine-tuned model

In [ ]:
#import locale
#locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
model = model.merge_and_unload()

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [ ]:
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/tokenizer.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')